# Toy example for Radix-based for all conditions

In [1]:
SCALE_CROWDING=False
PREVENT_ZERO = True

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7f8975237ef0>)


### Try optimizing using radix for one condition first

In [4]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')
mdl_ref = ijomc
keff0 = 1./65/3600
#crowding_bound = 0.0003
crowding_bound0 = 0.001
crowding_bound = crowding_bound0

not_crowded = ['ATPM']
rxns_c = [r for r in ijomc.reactions if all([m.compartment=='c' for m in r.metabolites.keys()]) and 'BIOMASS' not in r.id and r.id not in not_crowded]
crowding_dict = {rxn:keff0 for rxn in rxns_c}
#----------------------------------------

# Temporarily add crowding constraint for the duality gap constraint
crowding = Constraint('crowding')
crowding._bound = crowding_bound
crowding._constraint_sense = 'L'
for rxn,keff in iteritems(crowding_dict):
    rxn.add_metabolites({crowding:keff})

In [5]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [6]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

N_CONDS = len(conds)

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])
df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]
#df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'])]

## Need to allow higher growth for acetate with higher uptake rate

In [7]:
df_conds.loc[ df_conds.lb <= -10, 'lb'] = -20

In [8]:
stacker = StackOptimizer()
stacker.stack_models(ijomc, df_conds)

In [9]:
stacker.model.optimize(solver='gurobi')

<Solution 5.60 at 0x7f8914f2e410>

In [10]:
# stacker.model_dict['glucose'].optimize()
# print stacker.model_dict['glucose'].reactions.query('BIOMASS')[0].x
# print stacker.model_dict['glucose'].reactions.query('EX_glc')[0].x
# print stacker.model_dict['glucose'].reactions.query('EX_fru')[0].x

# stacker.model_dict['acetate'].optimize()
# print stacker.model_dict['acetate'].reactions.query('BIOMASS')[0].x
# print stacker.model_dict['acetate'].reactions.query('EX_glc')[0].x
# print stacker.model_dict['acetate'].reactions.query('EX_ac')[0].x
for mdl_ind,mdl in iteritems(stacker.model_dict):
    mdl.optimize()
    print('%s. mu=%g.' % (mdl_ind, mdl.solution.f))

malate. mu=0.786489.
fructose. mu=1.79057.
succinate. mu=0.840134.
acetate. mu=0.389313.
glucose. mu=1.79057.


In [11]:
for mdl_ind,mdl in iteritems(stacker.model_dict):
    opt = Optimizer(mdl)
    gapi = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)

In [12]:
stacker.model.optimize(solver='gurobi')

<Solution 11.19 at 0x7f8914f2e650>

In [13]:
import numpy as np

radix = 2.
print('Radix:',radix)
powers = np.arange(-3,4)
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Discretize crowding coefficients into radix
#crowding_p = gap.metabolites.crowding
#crowding_cons = stacker.model.metabolites.query('crowding')

var_cons_dict = {}
# Get the group ID from reference model
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
for rxn_ref in crowding_ref.reactions:    
    for mdl_ind, mdl in iteritems(stacker.model_dict):
        crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
        var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
        rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
        # Get the coefficient in the dual
        cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
        a0 = rxn_p.metabolites[crowding_p]
        if var_cons_dict.has_key(rxn_ref.id):
            var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        else:
            var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]

('Radix:', 2.0)
('Powers:', array([-3, -2, -1,  0,  1,  2,  3]))
('Digits:', [1.0])


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:9 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [14]:
# %lprun -f opt.to_radix opt.to_radix(gap, var_cons_dict, radix, powers, digits_per_power, prevent_zero=True)

In [15]:
%time opt.to_radix(stacker.model, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)

CPU times: user 442 ms, sys: 14 ms, total: 456 ms
Wall time: 435 ms


[1.0]

### Try scaling
if SCALE_CROWDING:
    crowding = stacker.model.metabolites.crowding
    bound0  = crowding._bound
    f_scale = crowding._bound
    crowding._bound = bound0 / f_scale
    for rxn in crowding.reactions:
        rxn._metabolites[crowding] = rxn._metabolites[crowding]/f_scale

In [16]:
stacker.model.reactions.binary_ACALD00

<Variable binary_ACALD00 at 0x7f8914252dd0>

In [17]:
for rxn in stacker.model.reactions:
    rxn.objective_coefficient = 0.

for group_id in var_cons_dict.keys():
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):
            yid = 'binary_%s%s%s'%(group_id,k,l)
            y   = stacker.model.reactions.get_by_id(yid)
            ### PREFER pwr=0, digit=1
            if pwr==0 and digit==1:
                #print('Preferring a0')
                y.objective_coefficient = 0.
            else:
                y.objective_coefficient = 1.

In [ ]:
from cobra.solvers import gurobi_solver
from gurobipy import *
import numpy as np

F_TOL_FINAL = 0.02
F_TOL_START = 0.2
N_FS =6
F_TOLS = np.linspace(F_TOL_START, F_TOL_FINAL, N_FS)
F_TOLS

array([0.2  , 0.164, 0.128, 0.092, 0.056, 0.02 ])

### TOOD: bisection on F_TOL instead of linear

In [ ]:
%%time

import time

mu_id = 'BIOMASS_Ecoli_core_w_GAM'
toc = 0.
sol_dict = {}
for ind, F_TOL in enumerate(F_TOLS):
    # ind = 0
    # F_TOL = F_TOL_FINAL
    for mdl_ind,mdl in iteritems(stacker.model_dict):
        mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
        rxn_mu = mdl.reactions.get_by_id(mu_id+'_%s'%mdl_ind)    
        rxn_mu.lower_bound = mu_meas*(1-F_TOL)
        rxn_mu.upper_bound = mu_meas*(1+F_TOL)
        mu_crowd0 = np.nan
        print('Condition=%s. Initial mu_crowd=%g. Fitting within %g%% of measured: %g <= mu <= %g' % (mdl_ind, mu_crowd0, 100*F_TOL, rxn_mu.lower_bound, rxn_mu.upper_bound))

    milp = gurobi_solver.create_problem(stacker.model)
    milp.ModelSense = GRB.MINIMIZE
    milp.Params.IntFeasTol = 1e-9
    milp.Params.OutputFlag = 1
    # milp.Params.FeasibilityTol = 1e-9
    # milp.Params.OptimalityTol = 1e-9
    milp.Params.NodefileStart = 3   # Start writing nodes to disk if x GB RAM exceeded (for each thread)
    milp.Params.TimeLimit = 2*3600  # Time limit in seconds    
    #milp.Params.Method = 2 # Barrier
    #milp.Params.NodeMethod = 2 # Barrier slower for NodeMethod
    milp.Params.BarHomogeneous = 1
    tic = time.time()
    milp.optimize()
    toc = time.time()-tic

    # If feasible solution found, use it as MILP start
    if milp.SolCount > 0:
        print('#'*40)
        print('Setting MIP warm-start values.')
        for v in milp.getVars():
            v.Start = v.X
        sol_dict[ind] = {'x_dict':{v.VarName:v.X for v in milp.getVars()}, 'F_TOL':F_TOL, 'time':toc}

Condition=malate. Initial mu_crowd=nan. Fitting within 20% of measured: 0.352 <= mu <= 0.528
Condition=fructose. Initial mu_crowd=nan. Fitting within 20% of measured: 0.44 <= mu <= 0.66
Condition=succinate. Initial mu_crowd=nan. Fitting within 20% of measured: 0.368 <= mu <= 0.552
Condition=acetate. Initial mu_crowd=nan. Fitting within 20% of measured: 0.2048 <= mu <= 0.3072
Condition=glucose. Initial mu_crowd=nan. Fitting within 20% of measured: 0.592 <= mu <= 0.888
Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Changed value of parameter NodefileStart to 3.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter TimeLimit to 7200.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Changed value of parameter BarHomogeneous to 1
   Prev: -1  Min: -1  Max: 1  Default: -1
Optimize a model with 14669 rows, 5486 columns and 40452 nonzeros
Variable types: 5150 continuous, 336 integer (0 binary)
Coefficient statistics:
  Ma

In [ ]:
[(k, s['F_TOL'], s['time']) for k,s in iteritems(sol_dict)]

In [ ]:
best_ind = len(sol_dict)-1
best_ind

In [ ]:
x_dict = {r.id:sol_dict[best_ind]['x_dict'][str(i)] for i,r in enumerate(stacker.model.reactions)}
F_TOL = sol_dict[best_ind]['F_TOL']

In [ ]:
#sol = gurobi_solver.format_solution(milp, stacker.model)
#yopt = [sol.x_dict[rxn.id] for rxn in stacker.model.reactions.query('binary_')]
yopt = [x_dict[rxn.id] for rxn in stacker.model.reactions.query('binary_')]
print('Number of non-zero binaries: %g' % sum(yopt))
for mdl_ind,mdl in iteritems(stacker.model_dict):
    dfi = df_meas[ df_meas.substrate==mdl_ind]
    mu_measi = dfi.growth_rate_1_h.iloc[0]
    rxn_mu = stacker.model.reactions.get_by_id(mu_id+'_%s'%mdl_ind)
    muopt = x_dict[rxn_mu.id]
    muL = mu_measi*(1-F_TOL)
    muU = mu_measi*(1+F_TOL)
    print('%s: Initial mu_crowd=%g. Fitted within %g%%: %g <= %g <= %g' % (mdl_ind, mu_crowd0, 100*F_TOL, muL, muopt, muU))

In [ ]:
# Fitted parameters
kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = x_dict[yid]            
            if abs(y)>1e-10:
                print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))
#                 print yid, y
#             if abs(y)<1e-10:
#                 print yid, y
                pass
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

In [ ]:
kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
kfit_changed

### Plug back in to be sure

In [ ]:
#----------------------------------------
# Starting from basal model
csrcs = df_conds.cond.unique()
for csrc in csrcs:
    ijofit = load_json_model('/home/laurence/ME/models/BiGG_M/json/e_coli_core.json')    
    crowding = Constraint('crowding')
    crowding._bound = crowding_bound0
    crowding._constraint_sense = 'L'
    
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit})
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    
    # Get unfit
    for rxn in ijofit.metabolites.crowding.reactions:
        rxn._metabolites[crowding] = a0
    ijofit.optimize()
    mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    
    err = 100*(mu_fiti - mu_measi)/mu_measi
    print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g). Error=%.3g%%' % (csrc, mu_measi, mu_fiti, mu_unfiti, err))
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))